In [1]:
import os
import urllib
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import numpy as np

In [2]:
import glob
from osgeo import gdal
import rasterio as rio
import rasterio.plot
from rasterio.enums import Resampling
#import ospybook as pb

In [3]:
%matplotlib inline

In [4]:
dtm_fns = glob.glob('data/**/*.tif',recursive=True)
dtm_fns

['data/mcc_dem_3p0m_agg_TUOtrimmed_MANUAL.tif']

In [5]:
dtm_fn = dtm_fns[0] # change which of the two tif files to look at
imgdir = 'output/'
s='_'.join(dtm_fn.split('_')[7:8])
out_fn = os.path.join(imgdir,'ASO_3m_dtm_USCATM_'+s)
out_fn

'output/ASO_3m_dtm_USCATM_'

In [6]:
!gdalinfo $dtm_fn

Driver: GTiff/GeoTIFF
Files: data/mcc_dem_3p0m_agg_TUOtrimmed_MANUAL.tif
Size is 17699, 16780
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 11N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 11N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-117,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
          

In [7]:
in_ds = gdal.Open(dtm_fn)

In [8]:
in_band = in_ds.GetRasterBand(1)
in_data = in_band.ReadAsArray()

In [9]:
#in_data[in_data == 0] = np.nan

In [10]:
# f,ax = plt.subplots()
# colorbar = ax.imshow(in_data);
# f.colorbar(colorbar)

In [11]:
# Function to save a new raster, from https://livebook.manning.com/book/geoprocessing-with-python/chapter-11/50

def make_raster(in_ds, fn, data, data_type, nodata=None):
    """Create a one-band GeoTIFF.

    in_ds     - datasource to copy projection and geotransform from
    fn        - path to the file to create
    data      - NumPy array containing data to write
    data_type - output data type
    nodata    - optional NoData value
    """
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(
        fn, in_ds.RasterXSize, in_ds.RasterYSize, 1, data_type)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())
    out_band = out_ds.GetRasterBand(1)
    if nodata is not None:
        out_band.SetNoDataValue(nodata)
    out_band.WriteArray(data)
    out_band.FlushCache()
    out_band.ComputeStatistics(False)
    return out_ds

In [12]:
# Attempt to make a function that creates slices, to then be utilized in a moving window function. From https://livebook.manning.com/book/geoprocessing-with-python/chapter-11/100

def make_slices(data, win_size):
    """Return a list of slices given a window size.
    data     - two-dimensional array to get slices from
    win_size - tuple of (rows, columns) for the moving window
    """
    rows = data.shape[0] - win_size[0] + 1
    cols = data.shape[1] - win_size[1] + 1
    slices = []
    for i in range(win_size[0]):
        for j in range(win_size[1]):
            slices.append(data[i:rows+i, j:cols+j])
            return slices

In [13]:
# Find standard deviation within the file? From https://livebook.manning.com/book/geoprocessing-with-python/chapter-11/103

slices = make_slices(in_data, (3,3))
stacked_data = np.ma.dstack(slices)

In [14]:
rows, cols = in_band.YSize, in_band.XSize

In [15]:
out_data = np.ones((rows, cols), np.int32) * -99

In [17]:
out_data

array([[-99, -99, -99, ..., -99, -99, -99],
       [-99, -99, -99, ..., -99, -99, -99],
       [-99, -99, -99, ..., -99, -99, -99],
       ...,
       [-99, -99, -99, ..., -99, -99, -99],
       [-99, -99, -99, ..., -99, -99, -99],
       [-99, -99, -99, ..., -99, -99, -99]], dtype=int32)

In [ ]:
out_data[1:-1, 1:-1] = np.std(stacked_data) #, 2

In [ ]:
make_raster(in_ds, out_fn, out_data, gdal.GDT_Int32, -99)
del in_ds